Step 1 Installing and importing necessary libraries

In [ ]:
import cv2
import numpy as np
import os
import csv
import time
from datetime import datetime
from win32com.client import Dispatch
from keras.models import model_from_json
from keras.preprocessing import image

Step 2 Designing the GUI and Loading the Pretrained Model

In [ ]:
def speak(str1):
    speak = Dispatch("SAPI.SpVoice")
    speak.Speak(str1)

# Load the FaceNet model
def load_facenet_model(json_path, weights_path):
    with open(json_path, "r") as json_file:
        model_json = json_file.read()
    facenet_model = model_from_json(model_json)
    facenet_model.load_weights(weights_path)
    print("FaceNet model loaded successfully from JSON and weights.")
    return facenet_model

# Paths to FaceNet model files
facenet_json_path = r"C:\Users\MY\DSP1\db\facenet_model.json"
facenet_weights_path = r"C:\Users\MY\DSP1\db\facenet_model_weights.h5"

# Initialize FaceNet model
facenet_model = load_facenet_model(facenet_json_path, facenet_weights_path)

# Path to Haar Cascade XML for face detection
facedetect = cv2.CascadeClassifier(r"C:\Users\MY\DSP1\db\haarcascade_frontalface_default.xml")

# Load background image for the GUI
imgBackground = cv2.imread(r"C:\Users\MY\yolov8\roboflow screenshot\background.png")
COL_NAMES = ['DATE','NAME', 'TIME']

Step 3 Real-Time Attendance System

In [ ]:
# Open the webcam
video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    if not ret:
        print("Error: Failed to capture image")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = facedetect.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in detected_faces:
        crop_image = frame[y:y + h, x:x + w, :]
        resized_img = cv2.resize(crop_image, (160, 160))  # Resize for FaceNet input
        img_array = image.img_to_array(resized_img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Generate embeddings using FaceNet
        embeddings = facenet_model.predict(img_array)

        # Match the embedding with stored embeddings
        output = "Recognized_Person"  # Replace with actual matching logic

        # Record timestamp
        ts = time.time()
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H:%M:%S")

        # Draw rectangle and name on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, str(output), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Prepare attendance record
        attendance = [str(output), str(timestamp)]

    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame", imgBackground)

    # Save attendance on key press
    k = cv2.waitKey(1)
    if k == ord('o'):  # Press 'o' to save attendance
        speak("Attendance Taken, Have a nice day!!")
        time.sleep(3)
        save_attendance_to_csv(attendance, date)

    if k == ord('q'):  # Press 'q' to quit
        break

# Release resources
video.release()
cv2.destroyAllWindows()

Step 4 Automatically Saving Attendance into CSV File

In [ ]:
def save_attendance_to_csv(attendance, date):
    filename = "Attendance/Attendance_" + date + ".csv"
    file_exists = os.path.isfile(filename)

    if file_exists:
        with open(filename, "a") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(attendance)
    else:
        with open(filename, "w") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(COL_NAMES)
            writer.writerow(attendance)
    print(f"Attendance saved to {filename}")

Data Visualization from Real-time database

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from datetime import datetime
import os

# Function to get the path of today's attendance CSV file
def get_csv_file():
    date = datetime.now().strftime("%d-%m-%Y")
    return f"Attendance/Attendance_{date}.csv"

# Initialize the figure with 3 subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))  # Create a 1x3 grid for the charts
plt.subplots_adjust(wspace=0.5)  # Add space between the charts

# Function to update the graphs dynamically
def update_graph(i):
    csv_file = get_csv_file()  # Get today's CSV file
    if os.path.isfile(csv_file):  # Check if the file exists
        data = pd.read_csv(csv_file)

        # Check if 'NAME:' column exists in the CSV file
        if 'NAME:' in data.columns:
            # Count attendance occurrences by employee names
            counts = data['NAME:'].value_counts()

            # Bar Chart
            axs[0].clear()
            axs[0].bar(counts.index, counts.values, color='skyblue')
            axs[0].set_title("Bar Chart - Attendance Statistics")
            axs[0].set_xlabel("Employee Names")
            axs[0].set_ylabel("Attendance Count")
            axs[0].tick_params(axis='x', rotation=45) 

            # Line Chart - Attendance Trend Over Time
            axs[0, 1].clear()
            axs[0, 1].plot(daily_counts.index, daily_counts.values, marker='o', color='green')
            axs[0, 1].set_title("Line Chart - Attendance Trend")
            axs[0, 1].set_xlabel("Date")
            axs[0, 1].set_ylabel("Total Attendance")
            axs[0, 1].tick_params(axis='x', rotation=45)

            # Key Metric (Gauge-like Plot) - Overall Attendance Rate
            axs[1, 1].clear()
            total_employees = 50  # Assume total employees
            attendance_rate = counts.sum() / total_employees * 100
            axs[1, 1].barh(["Attendance Rate"], [attendance_rate], color='orange')
            axs[1, 1].set_xlim(0, 100)
            axs[1, 1].set_title("Overall Attendance Rate")
            axs[1, 1].set_xlabel("Percentage")

# Use Matplotlib's animation to update the charts every 3 seconds
ani = animation.FuncAnimation(fig, update_graph, interval=3000)

# Display the visualization
plt.show()